In [ ]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy import stats
import statistics

# Configuraciones de Matplotlib
%matplotlib inline
plt.rcParams['figure.dpi'] = 110
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from statsmodels.tsa.arima.model import ARIMA
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from statsmodels.tsa.arima.model import ARIMA

# Estilo por defecto para notebooks.
sns.set_theme('notebook')

### **Regresión lineal**

In [ ]:
#Carga del archivo local previamente tratado y ajustado
lineal = pd.read_excel('kpisbase.xlsx',sheet_name='Hoja1')

In [ ]:
#Tenemos una variable como Interacciones que es la que vamos a predecir y esta la vamos a visualizar en línea de tiempo con la columna mes
lineal.head()

,Mes,Interacciones,INDICE
0,2019-01-01,7509,0.75
1,2019-02-01,2696,0.27
2,2019-03-01,8563,0.85
3,2019-04-01,14323,1.42
4,2019-05-01,5701,0.57


In [ ]:
#Generamos un subset para cambiar la columna mes en una variable numérica ascendente
data_subset = lineal[['Mes','Interacciones','INDICE']]
data_subset.head()

,Mes,Interacciones,INDICE
0,2019-01-01,7509,0.75
1,2019-02-01,2696,0.27
2,2019-03-01,8563,0.85
3,2019-04-01,14323,1.42
4,2019-05-01,5701,0.57


In [ ]:
data_subset['Mes'] = data_subset['Mes'].astype("category").cat.codes + 1
data_subset.head(10)
data_subset.tail()

,Mes,Interacciones,INDICE
43,44,5410,2.10
44,45,46888,2.43
45,46,91137,4.73
46,47,122599,6.37
47,48,99174,5.15


In [ ]:
# visualizamos los datos para identificar si es lineal

import plotly.express as px

fig = px.scatter(data_subset, x="Mes", y="Interacciones", color="INDICE", trendline="ols")

fig.update_traces(marker=dict(size=10   )
fig.show()


In [ ]:
X = data_subset[['Mes']]
y = data_subset['Interacciones']
model = LinearRegression()
model.fit(X, y)

# Mostramos el R2 del modelo
r2lineal = model.score(X, y)
print("R2:", r2lineal)

# Hacemos las siguientes 12 predicciones
future_months = np.arange(len(data_subset['Mes']) + 1, len(data_subset['Mes']) + 13)
future_predictions = model.predict(future_months.reshape(-1, 1))

# Mostramos las predicciones
print("Predicciones:", future_predictions)


R2: 0.2666879987566235
Predicciones: [57264.62322695 58443.54577363 59622.46832031 60801.39086699
 61980.31341366 63159.23596034 64338.15850702 65517.0810537
 66696.00360038 67874.92614705 69053.84869373 70232.77124041]


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names



In [ ]:
#Calculate Standart deviation of Interacciones

interacciones_std = statistics.stdev(data_subset['Interacciones'])
print("Standard deviation of Interacciones:", interacciones_std)


Standard deviation of Interacciones: 31960.353414210847


In [ ]:
# prompt: generate a new plot with fig and the newpredictions generated in future months use plotly

fig = px.scatter(data_subset, x="Mes", y="Interacciones", color="INDICE", trendline="ols")
fig.update_traces(marker=dict(size=10))

fig.add_scatter(x=future_months, y=future_predictions, mode='lines', name='Predicciones')
fig.update_layout(title='Predicciones de interacciones', xaxis_title='Mes', yaxis_title='Interacciones')
fig.show()


###**Regresión cuadrátrica**

In [ ]:
#Carga del archivo local previamente tratado y ajustado
cuadratica = pd.read_excel('tabla_sesiones2.xlsx',sheet_name='Sheet1')

In [ ]:
cuadratica.head(10)

,Fecha,Mes,Año,Sesiones
0,2018-01-31,1,2018,101
1,2018-02-28,2,2018,100
2,2018-03-31,3,2018,108
3,2018-04-30,4,2018,110
4,2018-05-31,5,2018,122
5,2018-06-30,6,2018,125
6,2018-07-31,7,2018,128
7,2018-08-31,8,2018,124
8,2018-09-30,9,2018,122
9,2018-10-31,10,2018,118


In [ ]:
#Generamos un subset para cambiar la columna mes en una variable numérica ascendente
data_subset2 = cuadratica[['Fecha','Mes','Año','Sesiones']]
data_subset2.head()

,Fecha,Mes,Año,Sesiones
0,2018-01-31,1,2018,101
1,2018-02-28,2,2018,100
2,2018-03-31,3,2018,108
3,2018-04-30,4,2018,110
4,2018-05-31,5,2018,122


In [ ]:


fig = px.scatter(data_subset2, x="Mes", y="Sesiones", color="Año", trendline="lowess")
fig.update_traces(marker=dict(size=10))
fig.show()


In [ ]:
#Calcullamos el error lineal
from sklearn.metrics import mean_squared_error

X2 = data_subset2[['Mes']]
y2 = data_subset2['Sesiones']
model2 = LinearRegression()
model2.fit(X2, y2)

y_pred2 = model2.predict(X2)

error_lineal = mean_squared_error(y2, y_pred2)




In [ ]:
#Calculamos RMSE Y MAE

import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error

y_pred2 = model2.predict(X2)

rmse = np.sqrt(mean_squared_error(y2, y_pred2))
mae = mean_absolute_error(y2, y_pred2)




In [ ]:
#calculamos R2

from sklearn.metrics import r2_score

r2_cuadratica = r2_score(data_subset2['Sesiones'], model2.predict(data_subset2[['Mes']]))




In [ ]:
#Imprimimos los valores que nos van a ayudar a determinar la fiabilidad del modelo
print("Error lineal:", error_lineal)
print("RMSE:", rmse)
print("MAE:", mae)
print("R2:", r2_cuadratica)

Error lineal: 28.564669127169118
RMSE: 5.3445925127336995
MAE: 4.394295519295518
R2: 0.5846397703410731


In [ ]:
#Calculaamos desiación estandar de la variable principal y si esta es mayor que los errores podemos tenerlo como referencia para saber que el modelo se ajusta a la gran mayoría de los datos.

sesiones_std = statistics.stdev(data_subset2['Sesiones'])
print("Standard deviation of Sesiones:", sesiones_std)


Standard deviation of Sesiones: 8.351014107263758


In [ ]:
# Generamos 12 predicciones, en este caso para 2024

import pandas as pd


# Create a list of months for 2024
months_2024 = [f"2024-{i+1:02d}-01" for i in range(12)]

# Create a dataframe with the months and year
df_2024 = pd.DataFrame({"Fecha": months_2024, "Mes": range(1, 13), "Año": 2024})

# Use the model to predict the number of sessions for each month in 2024
df_2024["Sesiones"] = model2.predict(df_2024[["Mes"]])

# Print the predictions
print(df_2024)


         Fecha  Mes   Año    Sesiones
0   2024-01-01    1  2024  108.647436
1   2024-02-01    2  2024  110.484266
2   2024-03-01    3  2024  112.321096
3   2024-04-01    4  2024  114.157925
4   2024-05-01    5  2024  115.994755
5   2024-06-01    6  2024  117.831585
6   2024-07-01    7  2024  119.668415
7   2024-08-01    8  2024  121.505245
8   2024-09-01    9  2024  123.342075
9   2024-10-01   10  2024  125.178904
10  2024-11-01   11  2024  127.015734
11  2024-12-01   12  2024  128.852564


In [ ]:
#merge data_subset2 with df_2024

import pandas as pd
data_subset2_merged = pd.concat([data_subset2, df_2024], ignore_index=True)
data_subset2_merged.tail(15)


,Fecha,Mes,Año,Sesiones
69,2023-10-31 00:00:00,10,2023,120.000000
70,2023-11-30 00:00:00,11,2023,128.000000
71,2023-12-31 00:00:00,12,2023,122.000000
72,2024-01-01,1,2024,108.647436
73,2024-02-01,2,2024,110.484266
74,2024-03-01,3,2024,112.321096
75,2024-04-01,4,2024,114.157925
76,2024-05-01,5,2024,115.994755
77,2024-06-01,6,2024,117.831585
78,2024-07-01,7,2024,119.668415


In [ ]:


fig = px.scatter(data_subset2_merged, x="Mes", y="Sesiones", color="Año", trendline="lowess")
fig.update_traces(marker=dict(size=10))
fig.show()

print("Predicciones para 2024:")
print(df_2024)


Predicciones para 2024:
         Fecha  Mes   Año    Sesiones
0   2024-01-01    1  2024  108.647436
1   2024-02-01    2  2024  110.484266
2   2024-03-01    3  2024  112.321096
3   2024-04-01    4  2024  114.157925
4   2024-05-01    5  2024  115.994755
5   2024-06-01    6  2024  117.831585
6   2024-07-01    7  2024  119.668415
7   2024-08-01    8  2024  121.505245
8   2024-09-01    9  2024  123.342075
9   2024-10-01   10  2024  125.178904
10  2024-11-01   11  2024  127.015734
11  2024-12-01   12  2024  128.852564


### **Regresión Exponencial**

In [ ]:
#Carga del archivo local previamente tratado y ajustado
exponencial = pd.read_excel('exponencial5.xlsx',sheet_name='Sheet1')

In [ ]:
exponencial.head()

,Fecha,Año,Mes,Sesiones
0,2019-01-31,2019,1,738.905610
1,2019-02-28,2019,2,803.119500
2,2019-03-31,2019,3,872.913836
3,2019-04-30,2019,4,948.773584
4,2019-05-31,2019,5,1031.225850


In [ ]:
exponencial.describe()

,Año,Mes,Sesiones
count,60.000000,60.000000,60.000000
mean,2021.000000,6.500000,20889.784161
std,1.426148,3.481184,26078.171699
min,2019.000000,1.000000,738.905610
25%,2020.000000,3.750000,2527.481949
50%,2021.000000,6.500000,8641.844509
75%,2022.000000,9.250000,29534.968666
max,2023.000000,12.000000,100895.121231


In [ ]:
#Generamos un subset para cambiar la columna mes en una variable numérica ascendente
data_subset3 = exponencial[['Fecha','Sesiones','Año','Mes']]
data_subset3.head()

,Fecha,Sesiones,Año,Mes
0,2019-01-31,738.905610,2019,1
1,2019-02-28,803.119500,2019,2
2,2019-03-31,872.913836,2019,3
3,2019-04-30,948.773584,2019,4
4,2019-05-31,1031.225850,2019,5


In [ ]:
fig = px.scatter(data_subset3, x="Fecha", y="Sesiones", color="Año", labels="Fit")
fig.update_traces(marker=dict(size=10))
fig.show()

In [ ]:
# prompt: calculate the differents error values for exponential model, and r2, and standart deviation

import numpy as np
#Calculate the different error values for exponential model

X3 = data_subset3[['Mes']]
y3 = data_subset3['Sesiones']
model3 = Regression()
model3.fit(X3, y3)

y_pred3 = model3.predict(X3)

error_lineal = mean_squared_error(y3, y_pred3)
rmse = np.sqrt(mean_squared_error(y3, y_pred3))
mae = mean_absolute_error(y3, y_pred3)

#Calculate R2

r2_exponencial = r2_score(data_subset3['Sesiones'], model3.predict(data_subset3[['Mes']]))

#Calculate standard deviation

sesiones_std = statistics.stdev(data_subset3['Sesiones'])

#Print the results

print("Error lineal:", error_lineal)
print("RMSE:", rmse)
print("MAE:", mae)
print("R2:", r2_exponencial)
print("Standard deviation of Sesiones:", sesiones_std)


NameError: name 'data_subset3' is not defined

In [ ]:
# prompt: generate 12 predictions for the year 2024 and generate a plot

import pandas as pd
# Create a list of months for 2024
months_2024 = [f"2024-{i+1:02d}-01" for i in range(12)]

# Create a dataframe with the months and year
df_2024 = pd.DataFrame({"Fecha": months_2024, "Mes": range(1, 13), "Año": 2024})

# Use the model to predict the number of sessions for each month in 2024
df_2024["Sesiones"] = model3.predict(df_2024[["Mes"]])

# Print the predictions
print(df_2024)

# Merge the predictions with the existing data
data_subset3_merged = pd.concat([data_subset3, df_2024], ignore_index=True)

# Plot the data
fig = px.scatter(data_subset3_merged, x="Fecha", y="Sesiones", color="Año", labels="Fit")
fig.update_traces(marker=dict(size=10))
fig.show()


         Fecha  Mes   Año      Sesiones
0   2024-01-01    1  2024  11472.249979
1   2024-02-01    2  2024  13184.528921
2   2024-03-01    3  2024  14896.807863
3   2024-04-01    4  2024  16609.086805
4   2024-05-01    5  2024  18321.365748
5   2024-06-01    6  2024  20033.644690
6   2024-07-01    7  2024  21745.923632
7   2024-08-01    8  2024  23458.202574
8   2024-09-01    9  2024  25170.481517
9   2024-10-01   10  2024  26882.760459
10  2024-11-01   11  2024  28595.039401
11  2024-12-01   12  2024  30307.318343


/usr/local/lib/python3.10/dist-packages/plotly/express/_core.py:1074: FutureWarning:

Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])

